In [135]:
import os
import sys
import time
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import cv2

In [332]:
class Tile():
    def __init__(self, img, index):
        self.img = img
        self.index = index
        self.neighbors = {
            'top': [],
            'bottom': [],
            'left': [],
            'right': []
        }
        
    def __str__(self):
        plt.figure(figsize = (2, 2))
        plt.imshow(self.img)
        plt.show()
        return "Tile object"
    
    def __getitem__(self, idx):
        return self.img[idx]
        
    def add_neighbor(self, direction, tile):
        self.neighbors[direction].append(tile)

In [333]:
def check_side(side1, side2):
    ratio = 0.8
    num_pixels = np.prod(side1.shape)
    threshold = ratio * num_pixels
    if np.sum(side1 == side2) > threshold:
        return True
    elif np.sum(side1[:-1] == side2[1:]) > threshold:
        return True
    elif np.sum(side1[1:] == side2[:-1]) > threshold:
        return True

In [334]:
def check_and_add(tile1, tile2):
    if check_side(tile1[0, :], tile2[-1, :]):
        tile1.add_neighbor('top', tile2)
        tile2.add_neighbor('bottom', tile1)
    if check_side(tile1[-1, :], tile2[0, :]):
        tile1.add_neighbor('bottom', tile2)
        tile2.add_neighbor('top', tile1)
    if check_side(tile1[:, 0], tile2[:, -1]):
        tile1.add_neighbor('left', tile2)
        tile2.add_neighbor('right', tile1)
    if check_side(tile1[:, -1], tile2[:, 0]):
        tile1.add_neighbor('right', tile2)
        tile2.add_neighbor('left', tile1)

In [335]:
img = cv2.imread('wfc-circuit-1.png')

start = 107
end = 121
tiles = []
idx = 0
for i in range(13):
    tile = Tile(img[start:end, :14, :], idx)
    idx += 1
    start += 16
    end += 16
    tiles.append(tile)

In [336]:
# rotate and flip tiles to augment data
N = len(tiles)
for i in range(N):
    tile = tiles[i]
    img = deepcopy(tile.img)
    img90 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    img180 = cv2.rotate(img, cv2.ROTATE_180)
    img270 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    if not np.all(img90 == img):
        new_tile = Tile(img90, idx)
        idx += 1
        tiles.append(new_tile)
    if not np.all(img180 == img) and not np.all(img180 == img90):
        new_tile = Tile(img180, idx)
        idx += 1
        tiles.append(new_tile)
    if not np.all(img270 == img) and not np.all(img270 == img90) and not np.all(img270 == img180):
        new_tile = Tile(img270, idx)
        idx += 1
        tiles.append(new_tile)

In [337]:
for i in range(len(tiles)):
    for j in range(i+1, len(tiles)):
        check_and_add(tiles[i], tiles[j])

In [321]:
def reduce_prob(canvas, row, col, choices, tiles):
    neighbor_choices = []
    valid_choices = []
    for i, j, direction in [[row-1, col, 'bottom'], [row+1, col, 'top'], [row, col-1, 'right'], [row, col+1, 'left']]:
        neighbor_tiles = []
        for tile_idx in choices[(i, j)]:
            neighbors = tiles[tile_idx].neighbors[direction]
            
        neighbor_choices.append()
    min_len = 1000
    for i in range(len(neighbor_choices)):
        length = len(neighbor_choices[i])
        if length < min_len:
            min_idx = i
            min_len = length
    for choice in neighbor_choices[i]:
        vote = 0
        for neighbor in neighbor_choices: # check if in possible neighbors list
            if choice in neighbor:
                vote += 1
        if vote == len(neighbor_choices):
            valid_choices.append(choice)
    choices[(row, col)] = valid_choices

In [322]:
rows = 10
cols = 10
canvas = np.zeros(shape = (rows, cols))
num_tiles = len(tiles)
choices = {}
for i in range(rows):
    for j in range(cols):
        choices[(i, j)] = np.arange(num_tiles).tolist()
indices = []
for i in range(rows):
    for j in range(cols):
        indices.append([i, j])
idx_list = np.arange(len(indices)).tolist()
idx = np.random.choice(idx_list)
idx_list.remove(idx)
row, col = indices[idx]
state = np.random.choice(choices[(row,  col)])
canvas[row,  col] = state
choices[(row, col)] = [state]

# compute new probability for 4 immediate neighbors
for i, j in [[row-1, col], [row+1, col], [row, col-1], [row, col+1]]:
    reduce_prob(canvas, i, j, choices)

In [325]:
canvas

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 34.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
for i in range(len(tiles)):
    print(f"Showing neighbors of tile {i}")
    print(tiles[i])
    print('-------------------------------')
    for key in tiles[0].neighbors:
        for j in range(len(tiles[i].neighbors[key])):
            print(key)
            print(tiles[i].neighbors[key][j])
    print('==========================================')